In [ ]:
print("ok")

ok


In [ ]:
# Install some required packages

!pip install pypdf2
!pip install google-cloud-aiplatform
!pip install google-cloud-storage

In [ ]:
from google.cloud import storage
from vertexai.language_models import TextEmbeddingModel
from google.cloud import aiplatform
import PyPDF2

import re
import os
import random
import json
import uuid


In [ ]:
%ls

stats.pdf


In [ ]:
# Initialize some variable

project="vertex-ai-459208"
location="us-central1"

pdf_path="stats.pdf"
bucket_name = "stats-content2024"
embed_file_path = "stats_embeddings.json"
sentence_file_path = "stats_sentences.json"
index_name="stats_index"


In [ ]:
# helper

def extract_sentences_from_pdf(pdf_path):
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        text = ""
        for page in reader.pages:
            if page.extract_text() is not None:
                text += page.extract_text() + " "
    sentences = [sentence.strip() for sentence in text.split('. ') if sentence.strip()]
    return sentences

In [ ]:
def generate_text_embeddings(sentences) -> list:
  # aiplatform.init(project=project,location=location)
  model = TextEmbeddingModel.from_pretrained("textembedding-gecko@001")
  embeddings = model.get_embeddings(sentences)
  vectors = [embedding.values for embedding in embeddings]
  return vectors

In [ ]:
def generate_and_save_embeddings(pdf_path, sentence_file_path, embed_file_path):
    def clean_text(text):
        cleaned_text = re.sub(r'\u2022', '', text)  # Remove bullet points
        cleaned_text = re.sub(r'\s+', ' ', cleaned_text).strip()  # Remove extra whitespaces and strip
        return cleaned_text

    sentences = extract_sentences_from_pdf(pdf_path)
    if sentences:
        embeddings = generate_text_embeddings(sentences)

        with open(embed_file_path, 'w') as embed_file, open(sentence_file_path, 'w') as sentence_file:
            for sentence, embedding in zip(sentences, embeddings):
                cleaned_sentence = clean_text(sentence)
                id = str(uuid.uuid4())

                embed_item = {"id": id, "embedding": embedding}
                sentence_item = {"id": id, "sentence": cleaned_sentence}

                json.dump(sentence_item, sentence_file)
                sentence_file.write('\n')
                json.dump(embed_item, embed_file)
                embed_file.write('\n')

In [ ]:
def upload_file(bucket_name,file_path):
    storage_client = storage.Client()
    bucket = storage_client.create_bucket(bucket_name,location=location)
    blob = bucket.blob(file_path)
    blob.upload_from_filename(file_path)

In [ ]:
def create_vector_index(bucket_name, index_name):
    lakeside_index = aiplatform.MatchingEngineIndex.create_tree_ah_index(
    display_name = index_name,
    contents_delta_uri = "gs://"+bucket_name,
    dimensions = 768,
    approximate_neighbors_count = 10,
    )

    lakeside_index_endpoint = aiplatform.MatchingEngineIndexEndpoint.create(
    display_name = index_name,
    public_endpoint_enabled = True
    )

    lakeside_index_endpoint.deploy_index(
    index = lakeside_index, deployed_index_id = index_name
    )

In [ ]:
generate_and_save_embeddings(pdf_path,sentence_file_path,embed_file_path)
upload_file(bucket_name,sentence_file_path)

PdfReadError: EOF marker not found

In [ ]:
create_vector_index(bucket_name, index_name)

FailedPrecondition: 400 Service account `service-969290635131@gcp-sa-aiplatform.iam.gserviceaccount.com` does not have `[storage.buckets.get, storage.objects.get]` IAM permission(s) to the bucket "stats-content2024". Please either copy the files to the Google Cloud Storage bucket owned by your project, or grant the required IAM permission(s) to the service account.

In [ ]:
from vertexai.language_models import TextEmbeddingModel
from google.cloud import aiplatform
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Part
import json
import os

In [ ]:
# project=”YOUR_GCP_PROJECT”
location="us-central1"
sentence_file_path = "stats_sentences.json"
index_name="stats_index" #Get this from the console or the previous step

In [ ]:
# aiplatform.init(project=project,location=location)
# vertexai.init()
model = GenerativeModel("gemini-pro")
lakeside_index_ep = aiplatform.MatchingEngineIndexEndpoint(index_endpoint_name="1376179539650019328")

In [ ]:
def generate_text_embeddings(sentences) -> list:
    model = TextEmbeddingModel.from_pretrained("textembedding-gecko@001")
    embeddings = model.get_embeddings(sentences)
    vectors = [embedding.values for embedding in embeddings]
    return vectors


def generate_context(ids,data):
    concatenated_names = ''
    for id in ids:
        for entry in data:
            if entry['id'] == id:
                concatenated_names += entry['sentence'] + "\n"
    return concatenated_names.strip()



def load_file(sentence_file_path):
  data = []
  with open(sentence_file_path,'r') as f:
    for line in f:
      entry = json.loads(line)
      data.append(entry)
  return data





In [ ]:
data=load_file(sentence_file_path)
data

In [ ]:
query=["what is correlation?"]

qry_emb=generate_text_embeddings(query)

In [ ]:
# qry_emb

In [ ]:
response = lakeside_index_ep.find_neighbors(
    deployed_index_id = index_name,
    queries = [qry_emb[0]],
    num_neighbors = 10
)

In [ ]:
matching_ids = [neighbor.id for sublist in response for neighbor in sublist]

context = generate_context(matching_ids,data)
prompt=f"Based on the context delimited in backticks, answer the query. ```{context}``` {query}"

chat = model.start_chat(history=[])
response = chat.send_message(prompt)
print(response.text)